In [1]:
# %pip install google pg8000 sqlalchemy cloud-sql-python-connector

In [2]:
import os

from google.cloud.sql.connector import Connector, IPTypes
import pg8000

import sqlalchemy
from dotenv import load_dotenv
load_dotenv()

def connect_with_connector() -> sqlalchemy.engine.base.Engine:
    """
    Initializes a connection pool for a Cloud SQL instance of Postgres.

    Uses the Cloud SQL Python Connector package.
    """
    # Note: Saving credentials in environment variables is convenient, but not
    # secure - consider a more secure solution such as
    # Cloud Secret Manager (https://cloud.google.com/secret-manager) to help
    # keep secrets safe.

    instance_connection_name = os.environ[
        "INSTANCE_CONNECTION_NAME"
    ]  # e.g. 'project:region:instance'
    db_user = os.environ["DB_USER"]  # e.g. 'my-db-user'
    db_pass = os.environ["DB_PASS"]  # e.g. 'my-db-password'
    db_name = os.environ["DB_NAME"]  # e.g. 'my-database'

    ip_type = IPTypes.PRIVATE if os.environ.get("PRIVATE_IP") else IPTypes.PUBLIC

    # initialize Cloud SQL Python Connector object
    connector = Connector(refresh_strategy="LAZY")

    def getconn() -> pg8000.dbapi.Connection:
        conn: pg8000.dbapi.Connection = connector.connect(
            instance_connection_name,
            "pg8000",
            user=db_user,
            password=db_pass,
            db=db_name,
            ip_type=ip_type,
        )
        return conn

    # The Cloud SQL Python Connector can be used with SQLAlchemy
    # using the 'creator' argument to 'create_engine'
    pool = sqlalchemy.create_engine(
        "postgresql+pg8000://",
        creator=getconn,
        # ...
    )
    return pool

In [13]:
import sqlalchemy

engine = connect_with_connector()

inspector = sqlalchemy.inspect(engine)
table_names = inspector.get_table_names()
print(table_names)

['snp', 'snp_characteristic_link', 'skincharacteristic', 'characteristic_condition_link', 'skincondition', 'condition_ingredient_link', 'ingredient']


In [10]:
from sqlalchemy import Table, Column, Integer, String, Text, MetaData, CheckConstraint
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, ForeignKey
from sqlalchemy import insert

# Database engine
engine = connect_with_connector()

# Metadata object
metadata = MetaData()

# Table Definition
SNP = Table(
    'snp',
    metadata,
    Column('snp_id', Integer, primary_key=True, autoincrement=True),
    Column('rsid', String, nullable=False, unique=True),
    Column('gene', String, nullable=False),
    Column('risk_allele', String, nullable=False),
    Column('effect', Text),
    Column('evidence_strength', String, CheckConstraint(
        "evidence_strength IN ('Strong', 'Moderate', 'Weak')")),
    Column('category', String, nullable=False)
)

# SkinCharacteristic table
SkinCharacteristic = Table(
    'skincharacteristic',
    metadata,
    Column('characteristic_id', Integer, primary_key=True, autoincrement=True),
    Column('name', String, nullable=False, unique=True),
    Column('description', Text),
    Column('measurement_method', Text)
)

# SkinCondition table
SkinCondition = Table(
    'skincondition',
    metadata,
    Column('condition_id', Integer, primary_key=True, autoincrement=True),
    Column('name', String, nullable=False, unique=True),
    Column('description', Text),
    Column('severity_scale', Text)
)

# Ingredient table
Ingredient = Table(
    'ingredient',
    metadata,
    Column('ingredient_id', Integer, primary_key=True, autoincrement=True),
    Column('name', String, nullable=False, unique=True),
    Column('mechanism', Text),
    Column('evidence_level', String, CheckConstraint("evidence_level IN ('Strong', 'Moderate', 'Weak')")),
    Column('contraindications', Text)
)

# SNP_Characteristic_Link table
SNP_Characteristic_Link = Table(
    'snp_characteristic_link',
    metadata,
    Column('snp_id', Integer, ForeignKey('snp.snp_id'), primary_key=True),
    Column('characteristic_id', Integer, ForeignKey('skincharacteristic.characteristic_id'), primary_key=True),
    Column('effect_direction', String, CheckConstraint("effect_direction IN ('Increases', 'Decreases', 'Modulates')")),
    Column('evidence_strength', String)
)

# Characteristic_Condition_Link table
Characteristic_Condition_Link = Table(
    'characteristic_condition_link',
    metadata,
    Column('characteristic_id', Integer, ForeignKey('skincharacteristic.characteristic_id'), primary_key=True),
    Column('condition_id', Integer, ForeignKey('skincondition.condition_id'), primary_key=True),
    Column('relationship_type', String)
)

# Condition_Ingredient_Link table
Condition_Ingredient_Link = Table(
    'condition_ingredient_link',
    metadata,
    Column('condition_id', Integer, ForeignKey('skincondition.condition_id'), primary_key=True),
    Column('ingredient_id', Integer, ForeignKey('ingredient.ingredient_id'), primary_key=True),
    Column('recommendation_strength', String, CheckConstraint("recommendation_strength IN ('First-line', 'Second-line', 'Adjuvant')")),
    Column('guidance_notes', Text)
)

metadata.create_all(engine)

In [14]:
import pandas as pd

def populate_table(table):
    """
    Populates a table with data from a CSV file.
    """
    # Load the CSV file into a DataFrame
    df = pd.read_csv(f'ALGORYTHM\\tables\\{table.name}.csv')

    # Insert the data into the table
    with engine.connect() as conn:
        for index, row in df.iterrows():
            insert_stmt = insert(table).values(row.to_dict())
            conn.execute(insert_stmt)
            
            
populate_table(SNP)        

In [20]:
from sqlalchemy import text

engine = connect_with_connector()

with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM SNP"))
    for row in result:
        print(row)

In [21]:
result.all()

[]

In [ ]:
RESU``

In [ ]:
query =

engine = connect_with_connector()
# Example usage
with engine.connect() as conn:
    result = conn.execute("SELECT 1")
    # for row in result:
    #     print(row)

ObjectNotExecutableError: Not an executable object: "\nCREATE TABLE IF NOT EXISTS SNP (\n    snp_id SERIAL PRIMARY KEY,\n    rsid VARCHAR NOT NULL UNIQUE,\n    gene VARCHAR NOT NULL,\n    risk_allele VARCHAR NOT NULL,\n    effect TEXT,\n    evidence_strength VARCHAR CHECK (evidence_strength IN ('Strong', 'Moderate', 'Weak')),\n    category VARCHAR NOT NULL\n);\n"